In [ ]:
import mlflow

# Initialize MLFlow client
client = mlflow.tracking.MlflowClient()

# External model URI (for example, on AWS S3 or another location)
external_model_uri = "s3://my-bucket/my-model"

# Register the model with a name, linking to the external model URI
model_details = client.create_registered_model(name="S3_ExternalModel")

# Create a new version of the registered model pointing to the external URI
client.create_model_version(name="S3_ExternalModel", source=external_model_uri, run_id=None)

print(f"Model version registered: {model_details}")

In [ ]:
import mlflow

# Initialize MLFlow client
client = mlflow.tracking.MlflowClient()

# External model URI 
external_model_uri = "https://huggingface.co/openai/whisper-large-v3-turbo/tree/main"

# Register the model with a name, linking to the external model URI
model_details = client.create_registered_model(name="Open_AIWhisper")

# Create a new version of the registered model pointing to the external URI
client.create_model_version(name="Open_AIWhisper", source=external_model_uri, run_id=None)

print(f"Model version registered: {model_details}")

In [ ]:
import mlflow

# Initialize MLFlow client
client = mlflow.tracking.MlflowClient()


# Register the model with a name, linking to the external model URI
model_details = client.create_registered_model(name="Shell1")

# Create a new version of the registered model pointing to the external URI
client.create_model_version(name="Shell1", source='', run_id=None)

print(f"Model version registered: {model_details}")

In [ ]:
import mlflow

# Initialize MLFlow client
client = mlflow.tracking.MlflowClient()


# Register the model with a name, linking to the external model URI
model_details = client.create_registered_model(name="Shell2")

# Create a new version of the registered model pointing to the external URI
client.create_model_version(name="Shell2", source=None, run_id=None)

print(f"Model version registered: {model_details}")

In [ ]:
import os
import mlflow

# Directory where the empty model file will be created
model_dir = "empty_model"
os.makedirs(model_dir, exist_ok=True)

# Path to the empty model file
empty_model_file_path = os.path.join(model_dir, "empty_model.txt")

# Create an empty file
with open(empty_model_file_path, 'w') as f:
    f.write("")  # Write nothing to create an empty file

# Start an MLFlow run
with mlflow.start_run() as run:
    # Log the empty model file as an artifact
    mlflow.log_artifact(empty_model_file_path, artifact_path="model")

    # Register the model
    model_uri = f"runs:/{run.info.run_id}/model"
    mlflow.register_model(model_uri=model_uri, name="EmptyModel")

print("Empty model registered successfully.")

In [ ]:
#
# Register ChatGPT endpoint
#

import mlflow.pyfunc
import requests

import pandas as pd
import requests
import mlflow.pyfunc

# Define the endpoint details
endpoint_url = "https://api.openai.com/v1/chat/completions"
api_key = OPENAI_API_KEY  #"<your_openai_api_key>"


class ChatGPTModelWrapper(mlflow.pyfunc.PythonModel):
    def __init__(self, endpoint_url, api_key):
        self.endpoint_url = endpoint_url
        self.api_key = api_key

    def predict(self, context, model_input):
        # Convert input to a list of strings (assumes DataFrame or Series input)
        if isinstance(model_input, pd.DataFrame):
            messages = model_input.iloc[:, 0].tolist()
        elif isinstance(model_input, pd.Series):
            messages = model_input.tolist()
        elif isinstance(model_input, list):
            messages = model_input
        else:
            raise ValueError("Unsupported input format for model_input")

        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json",
        }
        payload = {
            "model": "gpt-4",
            "messages": [{"role": "user", "content": message} for message in messages],
        }
        response = requests.post(self.endpoint_url, headers=headers, json=payload)
        
        # Raise an error for bad HTTP responses
        response.raise_for_status()

        # Extract the API response
        return [choice["message"]["content"] for choice in response.json()["choices"]]

In [ ]:
# Save the wrapped model to a local directory
model_path = "chatgpt_model"
mlflow.pyfunc.save_model(
    path=model_path,
    python_model=ChatGPTModelWrapper(endpoint_url, api_key),
)

In [ ]:
# Log and register the model
with mlflow.start_run() as run:
    mlflow.pyfunc.log_model(
        artifact_path="chatgpt_model",
        python_model=ChatGPTModelWrapper(endpoint_url, api_key),
        registered_model_name="ChatGPT_Model",
    )

In [ ]:
import mlflow.pyfunc
import pandas as pd

# Load the registered model
model = mlflow.pyfunc.load_model("models:/ChatGPT_Model/latest")

# Test predictions
input_data = pd.DataFrame(["Hello, how are you?"], columns=["prompt"])
result = model.predict(input_data)
print(result)